In [63]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    confusion_matrix
)

In [69]:
df=pd.read_csv('/Users/isaromobru/Desktop/FakeNews 10.57.24/proyecto_machine_learning_Fake_News/notebooks/processedstopword.csv')

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer
df['text'] = df['text'].fillna("")
df['title'] = df['title'].fillna("")
df['subject'] = df['subject'].fillna("")

# Aplicar TF-IDF a las columnas 'text' y 'title'
vectorizer = TfidfVectorizer(max_features=5000)  # Se pueden ajustar las características

# Concatenar 'text' y 'title' en un solo corpus para mantener coherencia en el vocabulario
combined_text = df['text'] + " " + df['title']
tfidf_matrix = vectorizer.fit_transform(combined_text)

# Convertir la matriz TF-IDF a un DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())


In [71]:
# Aplicar LabelEncoder a 'subject'
label_encoder = LabelEncoder()
df['subject_encoded'] = label_encoder.fit_transform(df['subject'])

# Agregar la columna codificada al DataFrame TF-IDF
tfidf_df['subject_encoded'] = df['subject_encoded']


In [72]:
tfidf_df

,00,000,10,100,11,12,120,13,130,14,...,zealand,zero,zika,zimbabwe,zinke,zone,zor,zuckerberg,zuma,subject_encoded
0,0.0,0.000000,0.057170,0.000000,0.032815,0.0,0.0,0.000000,0.0,0.00000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,6
1,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,6
2,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,6
3,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,6
4,0.0,0.000000,0.000000,0.000000,0.021817,0.0,0.0,0.000000,0.0,0.00000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44684,0.0,0.000000,0.027035,0.000000,0.031036,0.0,0.0,0.000000,0.0,0.00000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1
44685,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1
44686,0.0,0.012839,0.000000,0.016543,0.000000,0.0,0.0,0.000000,0.0,0.00914,...,0.0,0.010705,0.0,0.0,0.0,0.142962,0.0,0.0,0.0,1
44687,0.0,0.033452,0.000000,0.000000,0.000000,0.0,0.0,0.047085,0.0,0.00000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1


In [75]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

# Verificar si la columna 'label' está en el DataFrame
if 'label' in df.columns:
    # Agregar la columna 'target' al DataFrame TF-IDF
    tfidf_df['target'] = df['label']
    
    # Separar características (X) y etiquetas (y)
    X = tfidf_df.drop(columns=['target'])
    y = tfidf_df['target']
    
    # Dividir en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Definir clasificadores individuales
    modelxgb = XGBClassifier(
        n_estimators=150,
        learning_rate=0.1,
        max_depth=6,
        random_state=42
    )
    modelxgb.fit(X_train, y_train)
    
    modelsvc = SVC(
        probability=True,
        kernel='rbf',
        C=1.0,
        gamma='scale',
        random_state=42
    )
    modelsvc.fit(X_train, y_train)
    
    modelrl = LogisticRegression(
        max_iter=1000,
        C=1.0,
        random_state=42
    )
    modelrl.fit(X_train, y_train)
    
    # Definir lista de clasificadores para el Ensemble
    classifiers = [
        ('xgb', modelxgb),
        ('svc', modelsvc),
        ('lr', modelrl)
    ]
    
    # Ensemble Voting Classifier con soft voting
    ensemble_clf = VotingClassifier(
        estimators=classifiers,
        voting='soft'
    )
    
    # Entrenar modelo
    ensemble_clf.fit(X_train, y_train)
    
    # Predicciones
    y_pred = ensemble_clf.predict(X_test)
    
    # Métricas de evaluación detalladas
    print("Métricas de Rendimiento:")
    print(classification_report(y_test, y_pred))
    
    # Matriz de confusión
    print("\nMatriz de Confusión:")
    print(confusion_matrix(y_test, y_pred))
    
    # Retornar modelo, conjunto de prueba y etiquetas
    return ensemble_clf, X_test, y_test
else:
    model_results = "La columna 'label' no se encuentra en el DataFrame."
    return model_results

SyntaxError: 'return' outside function (3323783193.py, line 73)

In [76]:
def train_fake_news_model(tfidf_df, df):
    # Verificar si la columna 'label' está en el DataFrame
    if 'label' in df.columns:
        # Agregar la columna 'target' al DataFrame TF-IDF
        tfidf_df['target'] = df['label']
        
        # Separar características (X) y etiquetas (y)
        X = tfidf_df.drop(columns=['target'])
        y = tfidf_df['target']
        
        # Dividir en conjuntos de entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        
        # Definir clasificadores individuales
        modelxgb = XGBClassifier(
            n_estimators=150,
            learning_rate=0.1,
            max_depth=6,
            random_state=42
        )
        modelxgb.fit(X_train, y_train)
        
        modelsvc = SVC(
            probability=True,
            kernel='rbf',
            C=1.0,
            gamma='scale',
            random_state=42
        )
        modelsvc.fit(X_train, y_train)
        
        modelrl = LogisticRegression(
            max_iter=1000,
            C=1.0,
            random_state=42
        )
        modelrl.fit(X_train, y_train)
        
        # Definir lista de clasificadores para el Ensemble
        classifiers = [
            ('xgb', modelxgb),
            ('svc', modelsvc),
            ('lr', modelrl)
        ]
        
        # Ensemble Voting Classifier con soft voting
        ensemble_clf = VotingClassifier(
            estimators=classifiers,
            voting='soft'
        )
        
        # Entrenar modelo
        ensemble_clf.fit(X_train, y_train)
        
        # Predicciones
        y_pred = ensemble_clf.predict(X_test)
        
        # Métricas de evaluación detalladas
        print("Métricas de Rendimiento:")
        print(classification_report(y_test, y_pred))
        
        # Matriz de confusión
        print("\nMatriz de Confusión:")
        print(confusion_matrix(y_test, y_pred))
        
        # Retornar modelo, conjunto de prueba y etiquetas
        return ensemble_clf, X_test, y_test
    else:
        print("La columna 'label' no se encuentra en el DataFrame.")
        return None

In [77]:
# Suponiendo que ya tienes tfidf_df y df
modelo, X_test, y_test = train_fake_news_model(tfidf_df, df)

KeyboardInterrupt: 